In [1]:
import cudf
from elasticsearch_dsl import Search as S
from connect import connection
from connect import ElasticsearchIndex as es_idx
from elasticsearch import Elasticsearch as E
import pandas as pd
from elasticsearch_dsl import Q
import cupy as cp

In [2]:
xr = None
def x(results):
    xs = S().using(connection)
    xs = xs.query("match", type='ConPot')
    xs = xs.source(["src_ip"])
    xs = xs[0:results]
    xr = xs.execute()
    return xr

In [3]:
def y(results):
    nresults = results + 1
    z = x(nresults)
    nhit = 0
    df = cudf.DataFrame()
    xdf = cudf.DataFrame()
    for hit in z:
        #nhit = nhit + 1
        xdf = cudf.DataFrame({
        'srcip': hit.src_ip,
        })
        df = df.append(xdf)
    return(df)

In [4]:
def uniqueIPs(sampleSize):
    noisydata = y(sampleSize)
    cleandata = noisydata.drop_duplicates()
    return cleandata

In [5]:
testuips = uniqueIPs(1000)
print("Output redacted, uncomment the line below and comment this line to reveal output")
#print(testuips.head)

Output redacted, uncomment the line below and comment this line to reveal output


In [6]:
testuips.to_csv('./srcip.csv')

In [7]:
!cat ./srcip.csv | wc -l

171


In [8]:
zr = None
def z(typekeyword):
    zdf = cudf.DataFrame()
    scandf = cudf.DataFrame()
    zs = S().using(connection)
    zs = zs.query("match", type=typekeyword)
    zs = zs.source(["src_ip"])
    for hit in zs.scan():
        zdf = cudf.DataFrame({
            'srcip': hit.src_ip,
        })
        scandf = scandf.append(zdf)
    return scandf

In [9]:
zz = z('CitrixHoneypot')
print("Output redacted, uncomment the line below and comment this line to reveal output")
#print(zz.head)
msg = "Number of IP addresses before deduplication: " + str(len(zz))

Output redacted, uncomment the line below and comment this line to reveal output


In [10]:
zzz = zz.drop_duplicates()
print("Output redacted, uncomment the line below and comment this line to reveal output")
#print(zzz.head)
msg = "Unique IP Addresses: " + str(len(zzz))

Output redacted, uncomment the line below and comment this line to reveal output
